In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import io
import re

# ---------- CONFIG ----------
input_dir = './ModeladoETLDatos'
output_dir = './datamart'

COUNTRY_WEIGHTS = {'SPAIN': 10, 'OTHER_EU': 20, 'NON_EU': 45}

# Create output dir
os.makedirs(output_dir, exist_ok=True)

# ---------- cargar datasets-------------------
inv_header = pd.read_csv(os.path.join(input_dir, 'invoices_header.csv'),sep=";").drop_duplicates()
inv_prod = pd.read_csv(os.path.join(input_dir, 'invoices_products.csv'),sep=";").drop_duplicates()
products = pd.read_csv(os.path.join(input_dir, 'products.csv'),sep=";").drop_duplicates()
suppliers = pd.read_csv(os.path.join(input_dir, 'suppliers.csv'),sep=";").drop_duplicates()
daily_cur = pd.read_csv(os.path.join(input_dir, 'daily_currencies.csv'),sep=";").drop_duplicates()

#--------------------------------------------------------------------------------
for col in ['InboundDate','OrderDate', 'InvoiceDate']:
    if col in inv_header.columns:
        inv_header[col] = pd.to_datetime(inv_header[col], errors='coerce')

inv_header.rename(columns={'Supplier':'IDSupplier'}, inplace=True)

daily_cur['Date']=pd.to_datetime(daily_cur['Date'], errors='coerce')

inv_prod['Quantity'] = inv_prod['Quantity'].abs()
inv_prod['PurchasePrice (Unit)'] = inv_prod['PurchasePrice (Unit)'].abs()
inv_prod.rename(columns={'PurchasePrice (Unit)':'Price'}, inplace=True)

def print_info(df, title):
    buffer = io.StringIO()
    df.info(buf=buffer)
    print(f"\n============== {title} ============== \n{buffer.getvalue()}\n")


    print(df.describe())

    nulls = df.isnull().sum()
    nulls = nulls[nulls > 0]      # filtrar solo columnas con nulls

    if nulls.empty:
        print(f"\n✓ {title}: sin valores nulos")
    else:
        print(f"\n⚠ {title}: columnas con nulos")
        print(nulls)
    
    print()

print_info(inv_header, "invoice_header")
print_info(inv_prod, "invoice_products")
print_info(products, "products")
print_info(suppliers, "suppliers")
print_info(daily_cur, "daily_currencies")





============== invoice_header ============== 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5178 entries, 0 to 5177
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      5178 non-null   object        
 1   InboundDate  5178 non-null   datetime64[ns]
 2   IDSupplier   5178 non-null   object        
 3   OrderDate    5178 non-null   datetime64[ns]
 4   InvoiceDate  5178 non-null   datetime64[ns]
dtypes: datetime64[ns](3), object(2)
memory usage: 202.4+ KB


                         InboundDate                      OrderDate  \
count                           5178                           5178   
mean   2017-10-18 08:52:00.278099712  2017-09-18 04:03:20.231749888   
min              2014-05-01 00:00:00            2014-03-13 00:00:00   
25%              2016-02-02 00:00:00            2016-01-05 00:00:00   
50%              2017-09-01 00:00:00            2017-08-05 00:00:00   
75%            

In [ ]:
# ---------- DF-DATE ----------
min_date = inv_header['InvoiceDate'].min()
max_date = inv_header['InvoiceDate'].max()

df_date = pd.DataFrame({'date': pd.date_range(start=min_date, end=max_date, freq='D')})

df_date['date_id'] = df_date.date.dt.strftime('%Y%m%d').astype(int)
df_date['year'] = df_date.date.dt.year
df_date['month'] = df_date.date.dt.month
df_date['day'] = df_date.date.dt.day
df_date['week'] = df_date.date.dt.weekday
df_date['quarter'] = df_date.date.dt.quarter

# ---------- dim_supplier ----------
#Reino Unido y Suecia sean de la UE
EU_CODES = {
    'AT','BE','BG','HR','CY','CZ','DK','EE','FI','FR','DE','GR','HU','IE',
    'IT','LV','LT','LU','MT','NL','PL','PT','RO','SK','SI','ES','SE', 'GB',
}

def country_category(code):
    code = str(code).strip().upper()

    if code == 'ES':
        return 'SPAIN'
    elif code in EU_CODES:
        return 'OTHER_EU'
    else:
        return 'NON_EU'

def country_weight(code):
    category = country_category(code)
    return COUNTRY_WEIGHTS[category]

dim_supplier = suppliers.copy()

dim_supplier.rename(columns={'Close':'Rate_to_eur'}, inplace=True)
dim_supplier['Country_category'] = dim_supplier['Country'].apply(country_category)
dim_supplier['theoretical_lead_time'] = dim_supplier['Country'].apply(country_weight)

print_info(dim_supplier, "dim_supplier")

print(dim_supplier.groupby('Country_category')['Currency'].unique())

#-------------------dim_product-------
dim_product = products.copy()
dim_product['Product'] = dim_product['Product'].str.strip().str.upper()
dim_product = dim_product.drop_duplicates(subset=['Product'])

def extract_number(text):
    text = str(text)  # Convierte todo a string
    match = re.search(r'\d+', text)
    return int(match.group()) if match else 0  # Si no hay número, devuelve 0

def missing_values(df,col):
    # Extraer números reales de la columna Type (ignora nulos)
    print(sorted(df[col].unique().tolist(),key=extract_number))
    type_numbers = df[col].dropna().apply(lambda x: int(re.search(r'\d+', str(x)).group()))
    type_numbers = sorted(type_numbers)
    
    # Mostrar huecos
    full_range = range(min(type_numbers), max(type_numbers)+1)
    missing = sorted(set(full_range) - set(type_numbers))
    print(f"Secuencia faltante en la columna {col}: {missing}\n" )

missing_values(dim_product,'Division')
missing_values(dim_product,'Group')

dim_product['Product'] = dim_product['Product'].fillna('Unknown')
dim_product['Type'] = dim_product['Type'].fillna('Unknown')
dim_product['Division'] = dim_product['Division'].fillna('Division 8')
dim_product['Group'] = dim_product['Group'].fillna('Grupo 7')

dim_product = dim_product[['Product','Type','Division','Group']]

print_info(dim_product, "dim_product")

#-----------------dim_currency--------------
dim_currency = daily_cur[['Date','Currency','Close']].copy()
dim_currency.rename(columns={'Date':'InvoiceDate','Close':'Rate_to_eur'}, inplace=True)

print("Monedas de cambio",dim_currency['Currency'].unique())

dim_currency = dim_currency.sort_values(['InvoiceDate'])

print_info(dim_currency, "dim_currency")


============== dim_supplier ============== 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SupplierName           1224 non-null   object
 1   PaymentMethod          1217 non-null   object
 2   PaymentTerms           1218 non-null   object
 3   IDSupplier             1224 non-null   object
 4   Country                1224 non-null   object
 5   Currency               1224 non-null   object
 6   Country_category       1224 non-null   object
 7   theoretical_lead_time  1224 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 76.6+ KB


       theoretical_lead_time
count            1224.000000
mean               13.333333
std                 9.975440
min                10.000000
25%                10.000000
50%                10.000000
75%                10.000000
max                45.000000

⚠ dim_supplier: columnas

In [3]:
#==================================join header + products============================================
fact = inv_prod.merge(inv_header, on='Invoice', how='left')

# --- producto-proveedor con precio inválido (=0) ---
invalid_pairs = fact.loc[fact['Price'] == 0, ['Product','IDSupplier']].drop_duplicates()
print(f"Total de precios invalidos por producto-proveedor {len(invalid_pairs)}\n")

# --- calcular estadísticas para estos pares (invalid_pairs) con precios válidos (>0) ---
stats_by_ps = (
    fact[fact['Price'] > 0]
    .merge(invalid_pairs, on=['Product','IDSupplier'], how='inner')
    .groupby(['Product','IDSupplier'])['Price']
    .agg(
        median='median',
        mean='mean',
        std='std',
        min='min',
        max='max'
    )
    .reset_index()
)

print(f"Estadisticas para estos pares invalidos con precios validos historicos siendo un total: {len(stats_by_ps)}\n")
print(stats_by_ps)
# --- 4. Reemplazar precios == 0 usando la mediana correspondiente ---
fact = fact.merge(stats_by_ps, on=['Product','IDSupplier'], how='left')
fact.rename(columns={'median':'Price_median'}, inplace=True)

fact['Price_median'] = fact.apply(
    lambda row: row['Price_median'] if row['Price'] == 0 else np.nan,
    axis=1
)

print(f"Total de producto-proveedor que no tienen precio medio validos: {len(invalid_pairs)-len(stats_by_ps)}")
print(fact.loc[
    (fact['Price'] == 0) & (fact['Price_median'].isnull()),
    ['Product', 'IDSupplier', 'Price_median']
].drop_duplicates().reset_index())

fact

Total de precios invalidos por producto-proveedor 29

Estadisticas para estos pares invalidos con precios validos historicos siendo un total: 23

     Product    IDSupplier    median      mean       std       min       max
0   MP019110  PROV40000520  2.541834  2.531516  0.643919  1.296000  4.821117
1   MP024224  PROV40002220  0.342633  0.342633  0.130960  0.250030  0.435235
2   MP024225  PROV40000187  0.265075  0.265075  0.161892  0.150600  0.379549
3   MP024227  PROV40002220  0.162227  0.188056  0.082065  0.136140  0.354257
4   MP025134      PROV1597  1.030816  1.234303  0.604450  0.588000  1.906434
5   MP025228      PROV1485  1.399980  1.298983  0.459725  0.594000  1.841034
6    MP02532      PROV1485  1.139632  1.228762  0.473668  0.660000  1.950459
7    MP02542      PROV1485  1.230437  1.142839  0.332932  0.690000  1.713239
8    MP02559      PROV1485  1.581261  1.572429  0.498372  0.810000  2.484688
9    MP02700  PROV40001000  0.099889  0.103907  0.036685  0.052200  0.203479
10   MP

,Invoice,Quantity,Product,Price,Section,InboundDate,IDSupplier,OrderDate,InvoiceDate,Price_median,mean,std,min,max
0,FFCC141196,1031,MP04245,0.441840,Seccion A,2014-05-26,PROV1650,2014-05-10,2014-05-29,NaN,NaN,NaN,NaN,NaN
1,FFCC141197,1931,MP04227,1.680570,Seccion D,2014-05-20,PROV40000235,2014-04-13,2014-05-23,NaN,NaN,NaN,NaN,NaN
2,FFCC141198,360,MP02868,1.159200,Seccion D,2014-05-12,PROV1647,2014-04-27,2014-05-15,NaN,NaN,NaN,NaN,NaN
3,FFCC141198,240,MP02869,1.214400,Seccion E,2014-05-12,PROV1647,2014-04-27,2014-05-15,NaN,NaN,NaN,NaN,NaN
4,FFCC141199,18138,MP02700,0.124391,Seccion E,2014-05-19,PROV40001000,2014-03-24,2014-05-22,NaN,0.103907,0.036685,0.0522,0.203479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39797,FFCC212138,2376,SE3174,0.087000,Seccion D,2021-05-01,PROV1875,2021-03-27,2021-05-04,NaN,NaN,NaN,NaN,NaN
39798,FFCC212138,6245,SE31101,0.192382,Seccion F,2021-05-01,PROV1875,2021-03-27,2021-05-04,NaN,NaN,NaN,NaN,NaN
39799,FFCC212138,819,SE3144,0.043325,Seccion E,2021-05-01,PROV1875,2021-03-27,2021-05-04,NaN,NaN,NaN,NaN,NaN
39800,FFCC212138,1291,SE31293,0.109718,Seccion E,2021-05-01,PROV1875,2021-03-27,2021-05-04,NaN,NaN,NaN,NaN,NaN


In [ ]:
#================================== join fact + dim_supplier ============================================
fact = fact.merge(dim_supplier, on='IDSupplier', how='left')

#================================== join fact + dim_currency ============================================
fact = fact.merge(dim_currency, on=['InvoiceDate','Currency'], how='left')

if fact[fact['Currency']=='USD']['Rate_to_eur'].isnull().any():
    usd = dim_currency[dim_currency['Currency'] == 'USD'].copy()

    # reindex por rango continuo de fechas
    full_range = pd.date_range(start=usd['InvoiceDate'].min(), end=usd['InvoiceDate'].max(), freq='D')

    usd = usd.set_index('InvoiceDate').reindex(full_range)
    # forward fill (rellena huecos con el último valor conocido)
    usd['Rate_to_eur'] = usd['Rate_to_eur'].ffill()

    # renombrar el index a 'date'
    usd.index.name = 'InvoiceDate'

    usd = usd.reset_index()

    # Pivot para búsquedas
    pivot_usd = usd.set_index('InvoiceDate')['Rate_to_eur']

    mask_usd = fact['Currency'] == 'USD'

    fact.loc[mask_usd, 'Rate_to_eur'] = fact.loc[mask_usd, 'InvoiceDate'].map(pivot_usd)

print(fact[fact['Currency']=='USD']['Rate_to_eur'].isnull().sum())

# Convertir a EUR
fact['Rate_to_eur'] = fact['Rate_to_eur'].fillna(1)

# Reemplazar solo los valores <=0
fact['Total_amount_eur'] = fact.apply(
    lambda row: row['Price_median']*row['Quantity']*row['Rate_to_eur'] if row['Price'] == 0 else row['Price']*row['Quantity']*row['Rate_to_eur'] ,
    axis=1
)

fact['actual_lead_time_days'] = (fact['InboundDate'] - fact['OrderDate']).dt.days

fact['delay_days'] = (fact['actual_lead_time_days'] - fact['theoretical_lead_time']).clip(lower=0)

fact['on_time'] = np.where((fact['delay_days'] == 0), 1, 0)

fact['Year'] = fact['InvoiceDate'].dt.year

fact['YearMonth'] = fact['InvoiceDate'].dt.to_period('M')

fact.head(3)

0


,Invoice,Quantity,Product,Price,Section,InboundDate,IDSupplier,OrderDate,InvoiceDate,Price_median,...,Currency,Country_category,theoretical_lead_time,Rate_to_eur,Total_amount_eur,actual_lead_time_days,delay_days,on_time,Year,YearMonth
0,FFCC141196,1031,MP04245,0.44184,Seccion A,2014-05-26,PROV1650,2014-05-10,2014-05-29,NaN,...,EUR,SPAIN,10,1.0,455.536759,16,6,0,2014,2014-05
1,FFCC141197,1931,MP04227,1.68057,Seccion D,2014-05-20,PROV40000235,2014-04-13,2014-05-23,NaN,...,EUR,SPAIN,10,1.0,3245.180374,37,27,0,2014,2014-05
2,FFCC141198,360,MP02868,1.15920,Seccion D,2014-05-12,PROV1647,2014-04-27,2014-05-15,NaN,...,EUR,SPAIN,10,1.0,417.312000,15,5,0,2014,2014-05


In [5]:
daily_pivot_section = fact.pivot_table(
    index=fact['InvoiceDate'].dt.date,
    columns='Section',
    values='Total_amount_eur',
    aggfunc='sum',
    fill_value=0
)

daily_pivot_section.index = pd.to_datetime(daily_pivot_section.index)
daily_pivot_section['day_of_month'] = daily_pivot_section.index.day


monthly_day_summary = (
    daily_pivot_section
    .groupby('day_of_month')
    .sum()
    .sort_index()
)



In [7]:

#proveedor–producto

supplier_product_leadtime=fact.groupby(
    ['IDSupplier','Product'], as_index=False
).agg(
    num_invoices = ('Invoice', pd.Series.nunique),
    total_deliveries = ('on_time', lambda x: np.sum(~pd.isna(x))),
    on_time = ('on_time', lambda x: np.sum(x)), 
    avg_lead_time_real = ('actual_lead_time_days','mean'),
    total_amount_eur = ('Total_amount_eur','sum'),
)

supplier_product_leadtime['on_time_rate'] = (
    supplier_product_leadtime['on_time'] /
    supplier_product_leadtime['total_deliveries']
)

supplier_product_leadtime = supplier_product_leadtime.sort_values(
    ["num_invoices", "on_time_rate"], ascending=False
)

supplier_product_leadtime.drop(columns=['total_deliveries', 'on_time'], inplace=True)

#proveedor–producto anual lead time real

supplier_product_leadtime_year=fact.groupby(
    ['IDSupplier','Product','Year'], as_index=False
).agg(
    num_invoices = ('Invoice', pd.Series.nunique),
    total_deliveries = ('on_time', lambda x: np.sum(~pd.isna(x))),
    on_time = ('on_time', lambda x: np.sum(x)), 
    avg_lead_time_real = ('actual_lead_time_days','mean'),
    total_amount_eur = ('Total_amount_eur','sum'),
)

supplier_product_leadtime_year['on_time_rate'] = (
    supplier_product_leadtime_year['on_time'] /
    supplier_product_leadtime_year['total_deliveries']
)

supplier_product_leadtime_year = supplier_product_leadtime_year.sort_values(
    ["num_invoices", "on_time_rate"], ascending=False
)
supplier_product_leadtime_year.drop(columns=['total_deliveries', 'on_time'], inplace=True)



In [11]:
# ---- Guardar dimensiones ----
df_date.to_csv(f"{output_dir}/dim_date.csv", index=False)
dim_supplier.to_csv(f"{output_dir}/dim_supplier.csv",index=False)
dim_product.to_csv(f"{output_dir}/dim_product.csv",index=False)
dim_currency.to_csv(f"{output_dir}/dim_currency.csv", index=False)

# ---- Guardar fact ----
fact.to_csv(f"{output_dir}/fact_purchases.csv", index=False, sep=';', decimal=',')

# ---- Guardar agregados ----
monthly_day_summary.to_csv(f"{output_dir}/monthly_day_summary_section.csv", sep=';', decimal=',')

supplier_product_leadtime.to_csv(f"{output_dir}/supplier_product_leadtime.csv", index=False, sep=';', decimal=',')
supplier_product_leadtime_year.to_csv(f"{output_dir}/supplier_product_leadtime_year.csv", index=False, sep=';', decimal=',')

print("✔ Todos los CSV guardados correctamente en", output_dir)


✔ Todos los CSV guardados correctamente en ./datamart


In [10]:
supplier_product_leadtime.head(10)

,IDSupplier,Product,num_invoices,avg_lead_time_real,total_amount_eur,on_time_rate
5259,PROV40000520,MP019423,228,29.377510,4.486289e+05,0.076305
5264,PROV40000520,MP019450,181,28.824742,5.529718e+05,0.097938
5410,PROV40002330,MP019307,170,31.359551,7.325614e+05,0.084270
1094,PROV1441,MP019602,151,29.804878,8.688941e+05,0.085366
1652,PROV1485,P5092,126,28.305970,4.887464e+05,0.350746
5254,PROV40000520,MP019110,114,30.425000,2.687936e+05,0.066667
1707,PROV1485,P5220,112,30.266129,1.126106e+06,0.298387
1671,PROV1485,P5133,109,27.694915,7.985592e+05,0.372881
1511,PROV1485,P4169,107,28.504132,3.765958e+05,0.363636
1647,PROV1485,P5083,102,26.877358,4.303012e+05,0.386792
